In [1]:
import spotipy
from spotipy import util
from spotipy.client import SpotifyException
from collections import Counter
from gensim.utils import tokenize
import time
import tqdm
import json

In [2]:
CLIENT_ID = '9d4442847caf4c278a9625e51438e8aa'
CLIENT_SECRET = '13b8ef1376cf4b7e98e356f43c1ad9cc'
USER_ID = 'DOsinga'

In [3]:
token = util.prompt_for_user_token(USER_ID, '',
                                   client_id=CLIENT_ID, client_secret=CLIENT_SECRET,
                                   redirect_uri='http://127.0.0.1:8000/callback')
session = spotipy.Spotify(auth=token)




            User authentication requires interaction with your
            web browser. Once you enter your credentials and
            give authorization, you will be redirected to
            a url.  Paste that url you were directed to to
            complete the authorization.

        
Opened https://accounts.spotify.com/authorize?client_id=9d4442847caf4c278a9625e51438e8aa&response_type=code&redirect_uri=http%3A%2F%2F127.0.0.1%3A8000%2Fcallback in your browser


Enter the URL you were redirected to: http://127.0.0.1:8000/callback?code=AQA34FKxXeRgFfp1yAquo4j9chV3fwxxgrbrpxcJQ_KsYf-HpmcgcsmloEaCA7eh9coZxHNza43Qy52eYnZWMJXFUoZbq6Qbqinz-2Pns-CsSci6-4yfuymDtlaO6_WB7fLNLTvWqr4kF6D-iH8Kq4zz3lK5crB9kKkDGxJcECmN5Wn9GMLkhRUB88HMHFdTfoGcJov6




In [4]:
def find_playlists(session, w, max_count=5000):
    try:
        res = session.search(w, limit=50, type='playlist')
        while res:
            for playlist in res['playlists']['items']:
                yield playlist
                max_count -= 1
                if max_count == 0:
                    raise StopIteration
            tries = 3
            while tries > 0:
                try:
                    res = session.next(res['playlists'])
                    tries = 0
                except SpotifyException as e:
                    tries -= 1
                    time.sleep(0.2)
                    if tries == 0:
                        raise
    except SpotifyException as e:
        status = e.http_status
        if status == 404:
            raise StopIteration
        raise

for pl in find_playlists(session, 'summer'):
    break

pl

{'collaborative': False,
 'external_urls': {'spotify': 'https://open.spotify.com/playlist/37i9dQZF1DX1gRalH1mWrP'},
 'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX1gRalH1mWrP',
 'id': '37i9dQZF1DX1gRalH1mWrP',
 'images': [{'height': None,
   'url': 'https://pl.scdn.co/images/pl/default/2f02947f1a4acbd139ca82b3288c82e29b6b70bd',
   'width': None}],
 'name': 'Summer Hits',
 'owner': {'display_name': 'Spotify',
  'external_urls': {'spotify': 'https://open.spotify.com/user/spotify'},
  'href': 'https://api.spotify.com/v1/users/spotify',
  'id': 'spotify',
  'type': 'user',
  'uri': 'spotify:user:spotify'},
 'primary_color': None,
 'public': None,
 'snapshot_id': 'MTUzNzg4ODMxNiwwMDAwMDA0MTAwMDAwMTY2MTE0OTZiZTMwMDAwMDE2NTQ1MzYyZDEz',
 'tracks': {'href': 'https://api.spotify.com/v1/playlists/37i9dQZF1DX1gRalH1mWrP/tracks',
  'total': 80},
 'type': 'playlist',
 'uri': 'spotify:user:spotify:playlist:37i9dQZF1DX1gRalH1mWrP'}

In [5]:
word_counts = Counter({'a': 1})
playlists = {}
words_seen = set()
playlists = {}
count = 0
dupes = 0

In [6]:
while len(playlists) < 100000:
    for word, _ in word_counts.most_common():
        if not word in words_seen:
            words_seen.add(word)
            print('word>', word)
            for playlist in find_playlists(session, word):
                if playlist['id'] in playlists:
                    dupes += 1
                elif playlist['name'] and playlist['owner']:
                    playlists[playlist['id']] = {
                      'owner': playlist['owner']['id'],
                      'name': playlist['name'],
                      'id': playlist['id'],
                    }
                    count += 1
                    for token in tokenize(playlist['name'], lowercase=True):
                        word_counts[token] += 1
            break

word> a
retrying ...1secs
retrying ...1secs


/home/ohtamans/.pyenv/versions/3.6.7/envs/cookbook/lib/python3.6/site-packages/ipykernel_launcher.py:6: DeprecationWarning: generator 'find_playlists' raised StopIteration
  


word> is
word> this
word> the
word> of
retrying ...1secs
word> best
word> s
retrying ...1secs
word> hits
word> top
word> pop
retrying ...1secs
word> music
word> and
retrying ...1secs
word> playlist
retrying ...1secs
word> love
retrying ...1secs
retrying ...1secs
word> i
retrying ...1secs
retrying ...1secs
word> you
word> to
word> for
retrying ...1secs
word> in
word> me
retrying ...1secs
word> my
word> soundtrack
retrying ...1secs
word> all
word> it
retrying ...1secs
word> on
word> dance
retrying ...1secs
word> rap
word> party
word> mix
retrying ...1secs
word> hop
retrying ...1secs
retrying ...1secs


In [7]:
def track_yielder(session, playlist):
    res = session.user_playlist_tracks(playlist['owner'], playlist['id'],
              fields='items(track(id, name, artists(name, id), duration_ms)),next')
    while res:
        for track in res['items']:
            if track['track']:
                yield track['track']
        tries = 3
        while tries > 0:
            try:
                res = session.next(res)
                if not res or  not res.get('items'):
                    raise StopIteration
                tries = 0
            except SpotifyException as e:
                if 400 <= e.http_status <= 499:
                    raise StopIteration
                tries -= 1
                time.sleep(1)
                if tries == 0:
                    raise e


In [14]:
import os
import sqlite3

if os.path.isfile('data/songs.db'):
    os.remove('data/songs.db')
conn = sqlite3.connect('data/songs.db')
c = conn.cursor()
c.execute('CREATE TABLE songs (id text primary key, name text, artist text)')
c.execute('CREATE INDEX name_idx on songs(name)')

tracks_seen = set()
with open('data/playlists.ndjson', 'w') as fout_playlists:
    with open('data/songs_ids.txt', 'w') as fout_song_ids:
        for playlist in tqdm.tqdm(playlists.values()):
            fout_playlists.write(json.dumps(playlist) + '\n')
            track_ids = []
            for track in track_yielder(session, playlist):
                track_id = track['id']
                if not track_id:
                    continue
                if not track_id in tracks_seen:
                    c.execute("INSERT INTO songs VALUES (?, ?, ?)", 
                              (track['id'], track['name'], track['artists'][0]['name']))
                track_ids.append(track_id)
                tracks_seen.add(track_id)
            fout_song_ids.write(' '.join(track_ids) + '\n')
            conn.commit()
conn.commit()


  0%|          | 0/102412 [00:00<?, ?it/s]/home/ohtamans/.pyenv/versions/3.6.7/envs/cookbook/lib/python3.6/site-packages/ipykernel_launcher.py:17: DeprecationWarning: generator 'track_yielder' raised StopIteration

  0%|          | 45/102412 [00:04<2:35:25, 10.98it/s]Exception in thread Thread-6:
Traceback (most recent call last):
  File "/home/ohtamans/.pyenv/versions/3.6.7/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/home/ohtamans/.pyenv/versions/3.6.7/envs/cookbook/lib/python3.6/site-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/home/ohtamans/.pyenv/versions/3.6.7/lib/python3.6/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  1%|          | 522/102412 [01:03<3:25:54,  8.25it/s]

retrying ...2secs
retrying ...1secs


  1%|          | 589/102412 [01:13<3:32:32,  7.98it/s]

retrying ...2secs
retrying ...1secs


  1%|          | 649/102412 [01:23<3:37:31,  7.80it/s]

retrying ...1secs
retrying ...1secs


  1%|▏         | 1534/102412 [03:12<3:31:15,  7.96it/s]

retrying ...3secs
retrying ...1secs


  2%|▏         | 1597/102412 [03:22<3:32:51,  7.89it/s]


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/users/?limit=100&offset=0&fields=items%28track%28id%2C+name%2C+artists%28name%2C+id%29%2C+duration_ms%29%29%2Cnext#1bigsister/playlists/3PBun0fKZkXzcjD1y2ZHWV/tracks:
 Service not found